In [ ]:
! pip install google-generativeai
! pip install langchain-google-genai


In [3]:
import os

In [5]:
import google.generativeai as genai

In [7]:
genai.configure(api_key="<your_api_key>")
model = genai.GenerativeModel("gemini-1.5-flash")

In [8]:
response = model.generate_content("I will give you a movie review, you just need to tell me if it \
                                   is positve or negative review. Just output one word POSITIVE if the review \
                                   is has a positive sentiment and NEGATIVE if the review has a negative \
                                   sentiment. \
                                   Movie Review: `The movie had a very good story line and a powerful cast` ")
print(response.text)

POSITIVE



In [9]:
model.count_tokens("I will give you a movie review, you just need to tell me if it \
                                   is positve or negative review. Just output one word POSITIVE if the review \
                                   is has a positive sentiment and NEGATIVE if the review has a negative \
                                   sentiment. \
                                   Movie Review: `The movie had a very good story line and a powerful cast` ")

total_tokens: 72

In [6]:
import asyncio

async def generate_response():
    response = await model.generate_content_async("The opposite of hot is")
    print(response.text)



In [8]:
try:
    asyncio.get_running_loop()
    # If running inside an existing loop, use `create_task`
    task = asyncio.create_task(generate_response())
    await task
except RuntimeError:
    # Otherwise, run normally
    asyncio.run(generate_response())

Cold



In [9]:
asyncio.get_running_loop()

<_UnixSelectorEventLoop running=True closed=False debug=False>

In [10]:
await asyncio.create_task(generate_response())

Cold



In [10]:
import pandas as pd
import time

/Users/psubburaj/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [11]:
df = pd.read_csv("IMDB_Dataset.csv")

In [12]:
df_sample = df[0:10]

In [13]:
df_sample

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [71]:
response = model.generate_content("I will give you a movie review, you just need to tell me if it \
                                   is positve or negative review. Just output one word POSITIVE if the review \
                                   is has a positive sentiment and NEGATIVE if the review has a negative \
                                   sentiment. \
                                   Movie Review: `The movie had a very good story line and a powerful cast` ")
print(response.text)

POSITIVE



In [85]:
y_true = df_sample.sentiment

In [86]:
y_true

0    positive
1    positive
2    positive
3    negative
4    positive
5    positive
6    positive
7    negative
8    negative
9    positive
Name: sentiment, dtype: object

In [97]:
start = time.time()
for _, row in df_sample.iterrows():
    movie_review = row.review
    prompt = f"I will give you a movie review, you just need to tell me if it is positve or negative review. \
    Just output one word POSITIVE if the review is has a positive sentiment and \
    NEGATIVE if the review has a negative sentiment. \
    Movie Review: {movie_review}"
    response = model.generate_content(prompt)
    print(response.text)

end = time.time()
print(f"Took - {end - start} seconds")

POSITIVE

POSITIVE

POSITIVE

NEGATIVE

POSITIVE

POSITIVE

POSITIVE

NEGATIVE

NEGATIVE

POSITIVE

Took - 8.650074243545532 seconds


In [14]:
import asyncio

# Define the async function
async def get_sentiment(review):
    prompt = f"I will give you a movie review, you just need to tell me if it is positive or negative review. \
Just output one word POSITIVE if the review has a positive sentiment and \
NEGATIVE if the review has a negative sentiment. \
Movie Review: {review}"
    response = await model.generate_content_async(prompt)
    return response.text.strip()  # Return both the review and the sentiment



In [15]:
# Main function to run all tasks
async def main():
    tasks = [get_sentiment(review) for review in df_sample.review]
    results = await asyncio.gather(*tasks)  # Run tasks concurrently
    for sentiment in results:
        print(sentiment)

In [16]:
start = time.time()
# Handling the running event loop
try:
    # If there's already a running event loop
    asyncio.get_running_loop()
    task = asyncio.create_task(main())  # Schedule the task
    await task
except RuntimeError:
    # If no event loop is running
    asyncio.run(main())
    
end = time.time()
print(f"Took - {end - start} seconds")

POSITIVE
POSITIVE
POSITIVE
NEGATIVE
POSITIVE
POSITIVE
POSITIVE
NEGATIVE
NEGATIVE
POSITIVE
Took - 3.621511936187744 seconds


### Working with Images

In [19]:
import httpx
import os
import base64

image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"

image = httpx.get(image_path)

prompt = "Where is this picture taken from?"
response = model.generate_content([{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, prompt])

print(response.text)

That's a view of London, England, taken from a high vantage point, likely a tall building or perhaps even a helicopter.  The perspective shows a wide area encompassing several landmarks, including:

* **The Houses of Parliament and Big Ben:** These are prominently featured in the center of the image.
* **The London Eye:** Visible in the left portion of the photo.
* **The Shard:**  This skyscraper is visible in the background.

The overall layout and the presence of these landmarks confirm the location as London.  The height suggests the photographer was significantly elevated above street level.



In [21]:
image_path = "https://t3.ftcdn.net/jpg/07/59/61/80/360_F_759618035_d01Fw9bfXZxnXwWkpCgS9J6UpTieDSrJ.jpg"

image = httpx.get(image_path)

prompt = "What happens when I cut the thread attached?"
response = model.generate_content([{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, prompt])

print(response.text)

If you cut the thread attached to the red balloon, the balloon will float away, propelled by the buoyant force of the helium (or other lighter-than-air gas) inside it.  The rate at which it floats away will depend on several factors, including the amount of lift provided by the gas, the weight of the balloon, and any air currents present.

